In [4]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to C:\Users\Sajeel
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sajeel
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:

class InvertedIndex():
	def __init__(self):
		self.db={}

	def __str__(self):
		return "Keys:"+str(len(list(self.db.keys())))
	
	def stripSpecialChar(self,text):
		return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

	def preProcess(self,text):
		text = text.lower()
		text_tokens = word_tokenize(text)
		stopWords = set(stopwords.words('english'))
		validTokens = list(set(text_tokens) - set(stopWords))
		validTokens = [self.stripSpecialChar(x) for x in validTokens]
		validTokens = [x for x in validTokens if len(x) > 1]
		return validTokens

	def indexFile(self,file,fileId):
		tokens = self.preProcess(file)
		for i in tokens:
			if i in self.db:
				self.db[i].append(fileId)
			else:
				self.db[i] = [fileId]

	def save(self):
		json.dump(self.db, open("output.json", "w"))


In [11]:
allFiles = os.walk("stories")
filePaths = []
for i in allFiles:
	for j in i[2]:
		filePath = i[0] + "/" + j
		filePaths.append(filePath)

json.dump(filePaths, open("mapping.json", "w"))

index = InvertedIndex()

for i,filePath in enumerate(tqdm(filePaths)):
    try:
        file = open(filePath, encoding="utf8")
        read = file.read().replace('\n', ' ')    
    except Exception as e:
        file = open(filePath, encoding="Latin1")
        read = file.read().replace('\n', ' ')
    file.close()
    index.indexFile(read, i)
    index.save()

In [12]:
class Query():
    def __init__(self):
        pass

    def loadIndex(self,file):
        self.db=json.load(open(file))

    def OR(self,term1,term2):
        return term1.add(term2)
    
    def AND(self,term1,term2):
        return term1.intersect(term2)
    
    def ANDNOT(self,term1,term2):
        # temp=Response(set(sum(list(self.db.copy().values()),[])))
        temp=Response(np.arange(468))
        temp1=temp.diff(term2)
        return term1.intersect(temp1)
    def ORNOT(self,term1,term2):
        # temp=Response(set(sum(list(self.db.copy().values()),[])))
        temp=Response(np.arange(468))
        temp1=temp.diff(term2)
        temp2 = term1.intersect(term2)
        return temp1.add(temp2)


In [13]:
class Response():
    def __init__(self,data):
        self.data=set(data)

    def getMapping(self,file):
        self.mapping=json.load(open(file))
        for i in self.data:
            print(self.mapping[i])
    
    def __str__(self):
        return str(len(self.data))

    def add(self,response):
        return Response(set.union(self.data,response.data))
        
    def intersect(self,response):
        return Response(set.intersection(self.data,response.data))

    def diff(self,response):
        return Response(set.difference(self.data,response.data))
        

In [14]:
query=Query()
query.loadIndex("output.json")

In [37]:
queries=["stink","middle"]
output=Response(query.db[queries[0]])
for i in tnrange(1,len(queries)):
    curr=Response(query.db[queries[i]])
    output=query.AND(curr,output)

print(output)


11


In [23]:
output.getMapping("mapping.json")
output.data

stories/18.lws
stories/19.lws
stories/3gables.txt
stories/3student.txt
stories/3wishes.txt
stories/6ablemen.txt
stories/7voysinb.txt
stories/adler.txt
stories/aesop11.txt
stories/aesopa10.txt
stories/alad10.txt
stories/altside.hum
stories/aquith.txt
stories/archive
stories/beautbst.txt
stories/beyond.hum
stories/blabnove.hum
stories/blabnove.txt
stories/blackp.txt
stories/bookem.1
stories/bookem2
stories/bookem3
stories/breaks1.asc
stories/breaks2.asc
stories/breaks3.asc
stories/bruce-p.txt
stories/bulmrx.txt
stories/bulwer.lytton
stories/bumm
stories/bureau.txt
stories/burintrv.66
stories/burintrv.92
stories/cabin.txt
stories/candle.hum
stories/cardcnt.txt
stories/clon
stories/consumdr.hum
stories/contrad1.hum
stories/cooldark.sto
stories/cooldark.txt
stories/corcor.hum
stories/crazy.hum
stories/cum
stories/cybersla.txt
stories/dakota.txt
stories/darkness.txt
stories/deer.txt
stories/descent.poe
stories/diaryflf.txt
stories/domain.poe
stories/dopedenn.txt
stories/dskool.txt
stories/el

{5,
 6,
 8,
 11,
 12,
 15,
 18,
 22,
 26,
 27,
 31,
 33,
 39,
 41,
 49,
 54,
 59,
 60,
 61,
 70,
 71,
 72,
 76,
 77,
 78,
 79,
 87,
 94,
 96,
 97,
 98,
 100,
 103,
 106,
 107,
 112,
 116,
 117,
 118,
 119,
 120,
 123,
 124,
 126,
 127,
 129,
 133,
 134,
 135,
 140,
 141,
 142,
 145,
 150,
 153,
 154,
 162,
 163,
 169,
 172,
 174,
 183,
 190,
 193,
 196,
 199,
 200,
 202,
 206,
 210,
 212,
 214,
 215,
 221,
 224,
 225,
 226,
 236,
 238,
 239,
 240,
 242,
 252,
 257,
 270,
 272,
 275,
 277,
 282,
 286,
 287,
 290,
 300,
 303,
 304,
 310,
 312,
 314,
 328,
 333,
 341,
 348,
 356,
 359,
 366,
 368,
 370,
 378,
 380,
 391,
 394,
 395,
 398,
 405,
 406,
 407,
 409,
 413,
 417,
 418,
 419,
 420,
 431,
 433,
 435,
 449,
 451,
 453,
 459,
 466}

In [35]:
def no_comparisonsOR(term1, term2):
    first = list(set(query.db[term1]))
    first.sort()
    second = list(set(query.db[term2]))
    second.sort()
    i=0
    j=0
    count=0
    ans = []
    while(i<len(first) and j<len(second)):
        count+=1
        if(first[i]<second[j]):
            ans.append(first[i])
            i+=1          
        elif(first[i]>second[j]):
            ans.append(second[j])
            j+=1
        else:
            ans.append(first[i])
            i+=1
            j+=1
    while(i<len(first)):
        ans.append(first[i])
        i+=1
    while(j<len(second)):
        ans.append(second[j])
        j+=1
    return count    

In [40]:
def no_comparisonsAND(term1, term2):
    first = list(set(query.db[term1]))
    first.sort()
    second = list(set(query.db[term2]))
    second.sort()
    i=0
    j=0
    count=0
    ans = []
    while(i<len(first) and j<len(second)):
        count+=1
        if(first[i]<second[j]):
            i+=1          
        elif(first[i]>second[j]):
            j+=1
        else:
            ans.append(first[i])
            i+=1
            j+=1
    return count

print(no_comparisonsAND("stink","middle"))

123
